# IE525 Final Project

***
author: 
Zeyuan Pan  zeyuanp2@illinois.edu
Shuxiang Lei lei14@illinois.edu
***

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy.interpolate as spi
import scipy.sparse as sp
import scipy.linalg as sla
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import spsolve
import scipy.stats as stats
from numpy import mat
import math

We learn about a put option with stock price S = 100, strike price K = 100, interest rate of 0.025, dividend rate of 0, maturity of 0.642 years and volatility of 0.2, which has the following characteristics:

| Properties | Symbol | Value |
|-----------|:------------:|:---:|
| Stock price | $S$ |100 |
| exercise price | $K$ |100 |
| continuous interest rate | $r$ |0.025 |
| continuous dividend rate | $q$ |0 |
| volatility | $\sigma$ |0.2 |
| years to maturity |$T$ |0.642 |


In [4]:
#init_param
(S, K, r, q, T, sigma, option_type) = (100, 100, 0.025, 0.00, 231 / 360, 0.2, 'put')
(Smin, Smax, Ns, Nt) = (0, 4*np.maximum(S,K), 200, 200)
np.random.seed(1000)

class init_option():
    
    def __init__(self, S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt):
        self.S = S
        self.K = K
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.option_type = option_type
        self.is_call = (option_type[0].lower()=='c')
        self.omega = 1 if self.is_call else -1
        self.Smin = Smin
        self.Smax = Smax
        self.Ns = int(Ns)
        self.Nt = int(Nt)
        self.dS = (Smax-Smin)/Ns * 1.0
        self.dt = T/Nt*1.0
        self.Svec = np.linspace(Smin, Smax, self.Ns+1)
        self.Tvec = np.linspace(0, T, self.Nt+1)
        self.grid = np.zeros(shape=(self.Ns+1, self.Nt+1))
        
    def _set_terminal_condition_(self):
        self.grid[:, -1] = np.maximum(self.omega*(self.Svec - self.K), 0)
    
    def _set_boundary_condition_(self):
        tau = self.Tvec[-1] - self.Tvec;     
        DFq = np.exp(-q*tau)
        DFr = np.exp(-r*tau)

        self.grid[0,  :] = np.maximum(self.omega*(self.Svec[0]*DFq - self.K*DFr), 0)
        self.grid[-1, :] = np.maximum(self.omega*(self.Svec[-1]*DFq - self.K*DFr), 0)        
        
    def _set_coefficient__(self):
        drift = (self.r-self.q)*self.Svec[1:-1]/self.dS
        diffusion_square = (self.sigma*self.Svec[1:-1]/self.dS)**2
        
        self.l = 0.5*(diffusion_square - drift)
        self.c = -diffusion_square - self.r
        self.u = 0.5*(diffusion_square + drift)
        
    def _solve_(self):
        pass
    
    def _interpolate_(self):
        tck = spi.splrep( self.Svec, self.grid[:,0], k=3 )
        return spi.splev( self.S, tck )
        #return np.interp(self.S, self.Svec, self.grid[:,0])
    
    def price(self):
        self._set_terminal_condition_()
        self._set_boundary_condition_()
        self._set_coefficient__()
        self._set_matrix_()
        self._solve_()
        return self._interpolate_()

### Part a Using crank nicolson to price a European put vanilla

Using the Crank-Nicolson method, the discretized form of this equation for an European option would be:
$$V_j^n+1 - V_j^n = dt/2 * ((σ² * j² * (V_{j+1}^n - 2V_j^n + V_{j-1}^n) + r * j * (V_{j+1}^n - V_{j-1}^n)/2 - r * V_j^n) + (σ² * j² * (V_{j+1}^{n+1} - 2V_j^{n+1} + V_{j-1}^{n+1}) + r * j * (V_{j+1}^{n+1} - V_{j-1}^{n+1})/2 - r * V_j^{n+1}))
$$

The boundary conditions for V at S=0 and S=∞ will depend on whether the option is a call or a put. For a call option, V=0 when S=0 and V=S when S=∞. For a put option, V=K when S=0 and V=0 when S=∞, where K is the strike price. The initial condition at time t=0 is V=max(S-K, 0) for a call and V=max(K-S, 0) for a put.

And The subscripts j refers to the spatial steps, while the superscripts n refer to the time steps.

In [5]:
class CrankNicolsonEu(init_option):

    theta = 0.5
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1],  format='csc')
        self.I = sp.eye(self.Ns-1)
        self.M1 = self.I + (1-self.theta)*self.dt*self.A
        self.M2 = self.I - self.theta*self.dt*self.A
    
    def _solve_(self):           
        _, M_lower, M_upper = sla.lu(self.M2.toarray())        
        for j in reversed(np.arange(self.Nt)):
            
            U = self.M1.dot(self.grid[1:-1, j+1])
            
            U[0] += self.theta*self.l[0]*self.dt*self.grid[0, j] \
                 + (1-self.theta)*self.l[0]*self.dt*self.grid[0, j+1] 
            U[-1] += self.theta*self.u[-1]*self.dt*self.grid[-1, j] \
                  + (1-self.theta)*self.u[-1]*self.dt*self.grid[-1, j+1] 
            
            Ux = sla.solve_triangular( M_lower, U, lower=True )
            self.grid[1:-1, j] = sla.solve_triangular( M_upper, Ux, lower=False )

In [6]:
# (th-1, alpha, epsilon) = (0.5, 1.5, 1e-6)
euro_opt = CrankNicolsonEu(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt)
parta_answer=euro_opt.price()
print("Using the Crank Nicolson method to compute the European option, the price is:", parta_answer.round(4))

Using the Crank Nicolson method to compute the European option, the price is: 5.5574


### Part b Using crank nicolson to price an American put vanilla

In [7]:
class CrankNicolsonAm(init_option):

    def __init__(self, S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, lbd, epsilon):
        super().__init__(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt)
        self.theta = theta
        self.lbd = lbd
        self.epsilon = epsilon
        self.max_iter = 10*Nt
    
    def _set_matrix_(self):
        self.A = sp.diags([self.l[1:], self.c, self.u[:-1]], [-1, 0, 1], format='csc')
        self.I = sp.eye(self.Ns-1)
    
    def _solve_(self):           
        (theta, dt) = (self.theta, self.dt)
        payoff = self.grid[1:-1, -1]
        pastval = payoff.copy()
        G = payoff.copy()
        
        for j in reversed(np.arange(self.Nt)):
            counter = 0
            noBreak = 1
            newval = pastval.copy()
            
            while noBreak:
                counter += 1
                oldval = newval.copy()
                D = sp.diags( (G > (1-theta)*pastval + theta*newval).astype(int), format='csc' )
                z = (self.I + (1-theta)*dt*(self.A - self.lbd*D))*pastval + dt*self.lbd*D*G
                
                z[0] += theta*self.l[0]*dt*self.grid[0, j] \
                 + (1-theta)*self.l[0]*dt*self.grid[0, j+1] 
                z[-1] += theta*self.u[-1]*dt*self.grid[-1, j] \
                  + (1-theta)*self.u[-1]*dt*self.grid[-1, j+1] 
                                
                M = self.I - theta*dt*(self.A - self.lbd*D)
                newval = spsolve(M,z)
        
                noBreak = CrankNicolsonAm.trigger( oldval, newval, self.epsilon, counter, self.max_iter )
            
            pastval = newval.copy()
            self.grid[1:-1, j] = pastval
    
    @staticmethod
    def trigger( oldval, newval, tol, counter, maxIteration ):
        noBreak = 1
        if np.max( np.abs(newval-oldval)/np.maximum(1,np.abs(newval)) ) <= tol:
            noBreak = 0
        elif counter > maxIteration:
            print('The results may not converge.')
            noBreak = 0
        return noBreak

In [8]:
(theta, lbd, epsilon) = (0.5, 1e6, 1e-6)
amer_opt = CrankNicolsonAm(S, K, r, q, T, sigma, option_type, Smin, Smax, Ns, Nt, theta, lbd, epsilon)
partb_answer=amer_opt.price()
print("Using the Crank Nicolson method to compute the American option, the price is:",partb_answer.round(4))

Using the Crank Nicolson method to compute the American option, the price is: 5.6921


### Part C Computing the continuous down and in put barrier option

The Black-Scholes-Merton PDE:
$$
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + rS \frac{\partial V}{\partial S} - rV = 0
$$
Approximation of derivative terms:

$$
\frac{\partial V}{\partial t} \approx \frac{V_i^{n+1} - V_i^n}{\Delta t}
$$

$$
\frac{\partial V}{\partial S} \approx \frac{V_{i+1}^n - V_{i-1}^n}{2\Delta S}
$$

$$
\frac{\partial^2 V}{\partial S^2} \approx \frac{V_{i+1}^n - 2V_i^n + V_{i-1}^n}{\Delta S^2}
$$

Crank-Nicolson method:
$$
V_i^{n+1} - V_i^n = \frac{\Delta t}{2} \left[ \left( \frac{\sigma^2 S_i^2 (V_{i+1}^n - 2V_i^n + V_{i-1}^n)}{\Delta S^2} + rS_i \frac{V_{i+1}^n - V_{i-1}^n}{2\Delta S} - rV_i^n \right) + \left( \frac{\sigma^2 S_i^2 (V_{i+1}^{n+1} - 2V_i^{n+1} + V_{i-1}^{n+1})}{\Delta S^2} + rS_i \frac{V_{i+1}^{n+1} - V_{i-1}^{n+1}}{2\Delta S} - rV_i^{n+1} \right) \right]
$$

Boundary conditions for an up-and-out call option:
$$
V = 
\begin{cases}
\max(S - K, 0), & \text{if } S < B \\
0, & \text{if } S \geq B
\end{cases}
$$


In [9]:


H = 80      # Barrier level
def cal_barrier_put(S, K, r, q, T, sigma, H):
    m = 1000
    dS = 200 / m
    dT = T / m

    # Initialize matrices and arrays
    array = np.zeros((m - 1, m - 1))
    matrix = mat(array)
    price_a = np.zeros((m + 1, m + 1))
    price_m = mat(price_a)

    # Generate coefficients for matrix
    for i in range(m - 1):
        j = i + 1
        sigma2 = (sigma * j) ** 2
        x1 = 0.5 * dT * (r * j - sigma2)
        x2 = 1 + dT * (sigma2 + r)
        x3 = 0.5 * dT * (-r * j - sigma2)

        if i == 0:
            matrix[i, i] = x2
            matrix[i, i + 1] = x3
        elif i == m - 2:
            matrix[i, i - 1] = x1
            matrix[i, i] = x2
        else:
            matrix[i, i - 1] = x1
            matrix[i, i] = x2
            matrix[i, i + 1] = x3

    # Initialize price_m
    price_m[:, m] = 0
    price_m[:, 0] = K
    for i in range(m + 1):
        price_m[m, i] = max((K - dS * i), 0)
        for j in range(m + 1):
            if (dS * j > H):
                price_m[m, j] = 0

    # Calculate Bput
    inverse_m = matrix.I
    for i in range(m, 0, -1):
        price = price_m[i, 1:m]
        z = np.zeros((m - 1))
        z[0] = x1 * price_m[i - 1, 0]
        z[m - 2] = x3 * price_m[i - 1, m]
        price1 = inverse_m * (price - z).reshape((m - 1, 1))
        price1 = price1.reshape(1, m - 1)
        price_m[i - 1, 1:m] = price1

    i = np.round(S / dS, 0)
    return price_m[0, int(i)]


# partc_answer = barrier_option_price(S, K, r, q, T, sigma, H)
partc_answer=cal_barrier_put(S, K, r, q, T, sigma, H)
print("Barrier Option Price:", partc_answer.round(4))

Barrier Option Price: 1.9964


### Part D.1 use Black Scholes price to reconcile your PDE price with prices from part a and part c.

According to the **BS** formula, the analytical solution of the Euclidean option is


$$
V = e^{-rT} \cdot E\left[ \left[ \omega\left(S_T-K\right)\right]^+\right]
= \omega\cdot\left[e^{-qT}S_0\Phi(\omega\cdot d_+) - e^{-rT}K\Phi(\omega\cdot d_{-})\right] 
$$

    
where


$$
d_{\pm} = \frac{1}{\sigma\sqrt{T}}\ln\left(\frac{S_0e^{(r-q)T}}{K}\right)\pm\frac{\sigma\sqrt{T}}{2}
$$


The call option corresponds to $\omega = 1$ and the put option corresponds to $\omega = -1$.

In [10]:
def blackscholes( S0=100, K=100, r=0.025, q=0.00, T=231 / 360, sigma=0.2, omega=1 ):
    discount = np.exp(-r*T)
    forward = S0*np.exp((r-q)*T)
    moneyness = np.log(forward/K)
    vol_sqrt_T = sigma*np.sqrt(T)
    
    d1 = moneyness / vol_sqrt_T + 0.5*vol_sqrt_T
    d2 = d1 - vol_sqrt_T
    
    V = omega * discount * (forward*norm.cdf(omega*d1) - K*norm.cdf(omega*d2))
    return V

partd_answer = blackscholes(S, K, r, q,T, sigma,-1)
print("Using the BS model the option price for european option is ",partd_answer.round(4))
print("the part a answer is",parta_answer.round(4))

Using the BS model the option price for european option is  5.5697
the part a answer is 5.5574


The Analytic formula to price a Continuous down and in put barrier option:
$$
P=-SN(-x1)+Ke^{-rT}N(-x1+\sigma \sqrt{T})+S{(\frac{B}{S})}^{2\mu}(N(y)-N(y1))-Ke^{-rT}{(\frac{B}{S})}^{2\mu-2}(N(y-\sigma \sqrt{T})-N(y1-\sigma \sqrt{T}))
$$
in this equation we got
$$\mu=\frac{r+\sigma^2/2}{\sigma^2}, y=\frac{ln\frac{B^2}{KS}}{\sigma\sqrt{T}}+\mu\sigma\sqrt{T},y1=\frac{ln\frac{B}{S}}{\sigma\sqrt{T}}+\mu\sigma\sqrt{T},x1=\frac{ln\frac{S}{B}}{\sigma\sqrt{T}}+\mu\sigma\sqrt{T}$$

In [14]:
# Define the variables
B=80
# K=100
# Calculate mu, x, y

# Calculate mu, x1, y, y1
mu = (r + sigma**2 / 2) / sigma**2
x1 = (np.log(S / B) / (sigma * np.sqrt(T))) + mu * sigma * np.sqrt(T)
y = (np.log(B**2 / (K*S)) / (sigma * np.sqrt(T))) + mu * sigma * np.sqrt(T)
y1 = (np.log(B / S) / (sigma * np.sqrt(T))) + mu * sigma * np.sqrt(T)

# Calculate the terms
term1 = -S * norm.cdf(-x1)
term2 = K * np.exp(-r*T) * norm.cdf(-x1 + sigma * np.sqrt(T))
term3 = S * (B / S) ** (2 * mu) * (norm.cdf(y) - norm.cdf(y1))
term4 = -K * np.exp(-r*T) * (B / S) ** (2 * mu - 2) * (norm.cdf(y - sigma * np.sqrt(T)) - norm.cdf(y1 - sigma * np.sqrt(T)))

# Add all terms
P = term1 + term2 + term3 + term4
# print(P)

print("Using the analytical formula to calculate the option price for barrier option is ",P)
print("the part c answer is", partc_answer.round(4))

Using the analytical formula to calculate the option price for barrier option is  3.071768855752139
the part c answer is 1.9964


### Part D.2 Use trinomial tree to reconcile the price from part b.

To calculate the price movements, we use:

- Up: $S_{u} = Su$
- Down: $S_{d} = Sd$
- Flat: $S_{m} = Sm$

For a put option, the exercise value at each node is calculated as:

$$
\text{Exercise Value} = \max(K - S, 0)
$$

The discounted expected future value at each node (assuming risk-neutral probabilities $p$, $q$, and $1-p-q$ for up, down, and flat movements respectively) is given by:

$$
\text{Expected Future Value} = e^{-r\Delta t} \left[ p V_{\text{up}} + q V_{\text{down}} + (1-p-q) V_{\text{flat}} \right]
$$

where $V_{\text{up}}$, $V_{\text{down}}$, and $V_{\text{flat}}$ are the option values at the next time step for up, down, and flat movements respectively.

The value of the put option at each node is then the maximum of the exercise value and the discounted expected future value:

$$
V = \max \left( K - S, e^{-r\Delta t} \left[ p V_{\text{up}} + q V_{\text{down}} + (1-p-q) V_{\text{flat}} \right] \right)
$$

In the trinomial model, the probabilities $p$, $q$, and $1-p-q$ and the factors $u$, $d$, and $m$ are determined using the risk-free rate, the volatility of the stock, and the time step size.


In [12]:
def american_trinominal_model(S, T, K, r, q, sigma, call):
    dt = 1. / 360
    N = int(T / dt)
    mu = r - q - (sigma ** 2) / 2.0
    smax = 2 * abs(mu) * dt ** .5
    smax = max(smax, sigma * (2 ** .5))
    if smax == 0:
        return -9999
    M = int(5 * (N ** .5))
    C_ = np.empty(2 * M + 1, dtype=np.float64)
    pC_ = np.empty(2 * M + 1, dtype=np.float64)
    S_ = np.empty(2 * M + 1, dtype=np.float64, )
    p = float(0.5 * (sigma ** 2)) / (smax ** 2)
    p_u = p + 0.5 * mu * dt ** .5 / float(smax)
    p_m = 1 - 2 * p
    p_d = p - 0.5 * mu * dt ** .5 / float(smax)
    D = 1.0 / (1 + r * dt)
    E = math.exp(smax * dt ** .5)

    for j in range(0, len(S_)):
        if j == 0:
            S_[j] = S * math.exp(-M * smax * dt ** .5)
        else:
            S_[j] = S_[j - 1] * E
        if call == True:
            C_[j] = max(S_[j] - K, 0)
        else:
            C_[j] = max(K - S_[j], 0)

    for k in range(0, N):
        for j in range(1, 2 * M):
            pC_[j] = (p_u * C_[j + 1] + p_m * C_[j] + p_d * C_[j - 1]) * D
        pC_[0] = 2 * pC_[1] - pC_[2]
        pC_[2 * M] = 2 * pC_[2 * M - 1] - pC_[2 * M - 2]

        for n in range(0, 2 * M + 1):
            if call == True:
                C_[n] = max(pC_[n], max(S_[n] - K, 0))
            else:
                C_[n] = max(pC_[n], max(K - S_[n], 0))
    ret = C_[M]
    return ret

partd_answer2=american_trinominal_model(S=S, K=K, r=r, q=q, T=T, sigma=sigma, call=False)
print("Using the trinominal model the price is",partd_answer2.round(4))
print("The part b price is ", partb_answer.round(4))

Using the trinominal model the price is 5.7037
The part b price is  5.6921


### Part E Use a Monte Carlo pricer to reconcile with price from part a.

We still use the BS equations to calculate, ans we have $$S_T=\ S_0e^{\left(r-\frac{\sigma^2}{2}\right)T}+\sigma\sqrt{T} W,\mu=r$$
where W is a Brownian Motion.

In [13]:


def monte_carlo_european_put(S0, K, r, sigma, T, num_simulations):
    np.random.seed(42)  # For reproducibility
    num_timesteps = 10
    dt = T / num_timesteps

    total_payoff = 0
    for i in range(num_simulations):
        S = S0
        for j in range(num_timesteps):
            z = np.random.standard_normal()
            S *= np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

        total_payoff += max(K - S, 0)

    price = np.exp(-r * T) * total_payoff / num_simulations
    return price

# Parameters from part (a)

num_simulations = 10000
monte_carlo_price = monte_carlo_european_put(S, K, r, sigma, T, num_simulations)

print("Monte Carlo European Put price:", monte_carlo_price.round(4))

Monte Carlo European Put price: 5.6675
